# Clean Classes=Categories with missing annotations or images
version: 1

info: 
- clean_cats: Can be used to remove classes with missing annotations or images:
    0. If not mola.json : Manually add dataset descriptors when importing id, e.g. COCO(see below)
    1. method="save_images" (if you didn't do it already)
    2. don't alter the excel
- reorder_ids: It also reorders category id

author: nuno costa

In [78]:
from annotate_v5 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib.image import imread
from matplotlib.patches import Rectangle
import random

In [79]:
#Define root dir dependent on OS
rdir_dsets='D:/external_datasets/' #WARNING: DATASETS ROOT is OK?
rdir='D:/external_datasets/MOLA/'  
if str(platform.platform()).find('linux')>-1:
    dirdir_dsets=rdir_dsets.replace('D:/','/mnt/d/')
    rdir=rdir.replace('D:/','/mnt/d/')
print('OS: {}'.format(platform.platform()))
print('root datasets dir: {}'.format(rdir_dsets))
print('root dir: {}'.format(rdir))

OS: Windows-10-10.0.21292-SP0
root datasets dir: D:/external_datasets/
root dir: D:/external_datasets/MOLA/


In [92]:
#jsonfile
injsonfile="coco2017" #"split_mola_fix_equal/test"
molajson =  json.load(open(rdir+'annotations/'+injsonfile+'.json'))
for k in molajson:
    print(k, len(molajson[k]))

info 6
licenses 8
images 123287
annotations 1170251
categories 80


## 1. Import ids
#### #NOTE: work with ids and index so you can use numpy for faster operations

In [93]:
# datasets name and id
dset_l=[]
dset_l_id=[]
try:
    for d in molajson['datasets']:
        dset_l.append(d['name'])
        dset_l_id.append(d['id'])
except: #manually add for example for only COCO
    dset_l=['COCO']
    dset_l_id=[1]
print(dset_l, dset_l_id)

['COCO'] [1]


In [94]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in molajson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    try:
        cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
    except:
        cat_l_dset.append(dset_l[0])
#print(cat_l_id)

In [95]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in molajson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [96]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
for an in tqdm(molajson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    try:
        ann_dset.append(an['dataset'])
    except:
        ann_dset.append(dset_l_id[0])

100%|██████████████████████████████████████████████████████████████████| 1170251/1170251 [00:01<00:00, 785834.27it/s]


## 2. Find cleaners cat_ids
cleaners example
categories= [{name:cow, id:1, dataset:1},...,{name:cow, id:200, dataset:2},...,{name:cow, id:101, dataset:3}]}

In [97]:
#cleaners #TODO: SORT alphabetically
cleaners_l=[]
cleaners_l_catid=[]
cleaners_l_catdset=[]
cleaner_method="all_cats"
if cleaner_method=="all_cats": #Do for all category names, even with equal 
    cleaners_l=cat_l
    cleaners_l_catid=[[id] for id in cat_l_id]
    cleaners_l_catdset=[[dset] for dset in cat_l_dset]
    

print(cleaners_l[0:5])
print(cleaners_l_catid[0:5])
print(cleaners_l_catdset[0:5])
print(len(cleaners_l))
print(len(cleaners_l_catid))
print(len(cleaners_l_catdset))

['person', 'bicycle', 'car', 'motorcycle', 'airplane']
[[1], [2], [3], [4], [5]]
[['COCO'], ['COCO'], ['COCO'], ['COCO'], ['COCO']]
80
80
80


In [98]:
# get annotations cleaners
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
cleaners_l_imgid=[]
cleaners_l_bbox=[]
cleaners_l_dset=[]
for catids in tqdm(cleaners_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
    cleaners_l_imgid.append(l_imgid)
    cleaners_l_bbox.append(l_bbox)
    cleaners_l_dset.append(l_dset)

100%|████████████████████████████████████████████████████████████████████████████████| 80/80 [00:02<00:00, 35.73it/s]


## 3. Classes|categories to clean w/ EXCEL report

In [102]:
#INIT VARS
classtoclean_l=[]
classtoclean_l_catid=[]
method="save_images" #"save_images": to save new images and excel for manual inspection; "": already saved don't need to repeat and the excel are
datadir="cleaners/"+injsonfile+"/" #root folder to save cleaner method . #WARNING cleaners/original json that was used to save images and excel
folder=cleaner_method+'/' #folder to save images and exel 
showimage=False #show images
startidx=0 # start index of image to save from each dataset
imgnr=5 # total number of images to save from each dataset
imgstep='random' # step between images: int | 'random' - int steps between images; 'rand' gets random list
#paths
path=os.path.join(rdir,datadir,folder) #path to folder
assure_path_exists(path)
excelpath=path+cleaner_method+"_v1.xlsx"#path+cleaner_method+"_classtoclean_report.xlsx"#path to excel

In [100]:
#METHODS
if method=="save_images": # save images and excel to folder for manual edit
    df=pd.DataFrame({'cleaners_l': cleaners_l,'cleaners_l_catid': cleaners_l_catid, 'cleaners_l_catdset': cleaners_l_catdset, 'classtoclean_l': np.nan, 'classtoclean_l_catid':np.nan, 'rules':np.nan })
    df.loc[0, 'rules']="To fix classes: 1) You need to fill the column classtoclean_l and/or classtoclean_l_catid with the information from the respective cleaner columns; 2) When copy/pasting or changing, make sure the same structure maintains:  ['car', 'carrot'], [3, 52], beware of the spaces ['car', '  and always maintain the first class in the list;  3) You have 3 possibilities of filling the columns : 1-the 2 columns empty, meaning the row will not be used for classtoclean; 2-only one column empty, e.g. fill the classtotix_l row with the class labels from cleaners_l, then during the importing the classtoclean_l_catid is filled, and vice-versa; 3-If you want to change the name of the first class in the list,e.g ['car', 'carrot'] for ['automobile', 'carrot'] you need to provide the ids to classtoclean_l_catid."
    df['annotations_missing'] = np.empty((len(df), 0)).tolist()
    df['images_missing'] = np.empty((len(df), 0)).tolist()
    #save image for each cleaner
    for i, cleaner in enumerate(tqdm(cleaners_l)): #run for each cleaner category
        firstclass=cleaner
        if isinstance(firstclass, list): firstclass=firstclass[0] #first class
        print('\n>> '+firstclass+'...') #class
        classpath=os.path.join(path, firstclass) # path to folder for images of  firstclass
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        df=save_imgs(df, rdir_dsets, classpath, i, dset_l, cleaners_l, cleaners_l_catid, cleaners_l_bbox, cleaners_l_dset,
              cleaners_l_imgid, img_l, img_l_id, startidx=startidx, imgnr=imgnr, imgstep=imgstep, showimage=showimage)    
    df.to_excel(excelpath, index=False)   

  0%|                                                                                         | 0/80 [00:00<?, ?it/s]


>> person...
COCO


  1%|█                                                                                | 1/80 [00:00<01:07,  1.17it/s]


>> bicycle...
COCO



>>> Finding COCO bicycle...:   0%|                                                          | 0/7429 [00:00<?, ?it/s]

...

 98%|██████████████████████████████████████████████████████████████████████████████  | 78/80 [00:43<00:00,  2.04it/s]


>> hair drier...
COCO


 99%|███████████████████████████████████████████████████████████████████████████████ | 79/80 [00:43<00:00,  1.96it/s]


>> toothbrush...
COCO


100%|████████████████████████████████████████████████████████████████████████████████| 80/80 [00:44<00:00,  1.80it/s]


In [113]:
#IMPORT EXCEL MANUAL EDIT #WARNING: CHECK EXCEL FIRST (#NOTE: donte use classes with missing annotations and images)
df=pd.read_excel(excelpath)
classtoclean_df=df.loc[:,'classtoclean_l']
classtoclean_df_catid=df.loc[:,'classtoclean_l_catid']
cleaners_df_catid=df.loc[:,'cleaners_l_catid']
annotations_missing_df=df.loc[:,'annotations_missing']
images_missing_df=df.loc[:,'images_missing']
new_cat_l=copy.deepcopy(cat_l)
new_cat_l_id=copy.deepcopy(cat_l_id)
display(df)

# PARSE COLUMNS TO FIX
classtoclean_l=[]
classtoclean_l_catid=[]
cleaners_l_catid=cleaners_df_catid.tolist()
annotations_missing=annotations_missing_df.tolist()
images_missing=images_missing_df.tolist()
#convert strings to lists
for row, mx in enumerate(cleaners_l_catid):
    if isinstance(cleaners_l_catid[row], str): cleaners_l_catid[row]=convert_unicode(cleaners_l_catid[row], method='itemnum')
    if isinstance(annotations_missing[row], str): annotations_missing[row]=convert_unicode(annotations_missing[row], method='itemnum')
    if isinstance(images_missing[row], str): images_missing[row]=convert_unicode(images_missing[row], method='itemnum')


#remove classes with missing annotations and images 
removeidx_l=[]
remove_counter=0
remove_missings=True #remove classes with missing annotations or images
if remove_missings:
    for ir,catid in enumerate(new_cat_l_id):
        clean_idx = cleaners_l_catid.index(catid) #index of cat in excel cleaners_l
        if annotations_missing[clean_idx]==1 or images_missing[clean_idx]==1:
            #print(">> removing {},{}: annotations_missing={}, images_missing={} ".format(new_cat_l[ir],new_cat_l_id[ir],annotations_missing[clean_idx],images_missing[clean_idx] ))
            remove_counter+=1
            removeidx_l.append(ir)
            #NOTE: we assume (classtoclean_l_catid) are classes with no missing annotations and images
    removeitem_l=[new_cat_l[removeidx] for removeidx in removeidx_l] ##remove items instead of index #WARNING NECESSARY BECAUSE THE INDEX WILL CHANGE
    for removeitem in removeitem_l: new_cat_l.remove(removeitem) 
    removeitem_l=[new_cat_l_id[removeidx] for removeidx in removeidx_l] ##remove items instead of index #WARNING NECESSARY BECAUSE THE INDEX WILL CHANGE
    for removeitem in removeitem_l: new_cat_l_id.remove(removeitem)
print(">> remove_counter: ", remove_counter )
print(">> len(new_cat_l): ", len(new_cat_l) )
print(">> len(new_cat_l_id): ", len(new_cat_l_id) )


classtoclean_l=new_cat_l
classtoclean_l_catid= [[id] for id in new_cat_l_id] #needs to be a list


        

,mixers_l,mixers_l_catid,mixers_l_catdset,classtomix_l,classtomix_l_catid,rules,annotations_missing,images_missing
0,person,[1],['COCO'],NaN,NaN,To fix classes: 1) You need to fill the column...,[0],[0]
1,bicycle,[2],['COCO'],NaN,NaN,NaN,[0],[0]
2,car,[3],['COCO'],NaN,NaN,NaN,[0],[0]
3,motorcycle,[4],['COCO'],NaN,NaN,NaN,[0],[0]
4,airplane,[5],['COCO'],NaN,NaN,NaN,[0],[0]
...,...,...,...,...,...,...,...,...
75,vase,[86],['COCO'],NaN,NaN,NaN,[0],[0]
76,scissors,[87],['COCO'],NaN,NaN,NaN,[0],[0]
77,teddy bear,[88],['COCO'],NaN,NaN,NaN,[0],[0]
78,hair drier,[89],['COCO'],NaN,NaN,NaN,[0],[0]


>> remove_counter:  0
>> len(new_cat_l):  80
>> len(new_cat_l_id):  80


In [114]:
print('>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! \n')
fixempty=True
if fixempty:
    classtoclean_l=[x for x in classtoclean_l if str(x) != 'nan' and str(x) !='[]']
    classtoclean_l_catid=[x for x in classtoclean_l_catid if str(x) != 'nan' and str(x) !='[]']
print(len(classtoclean_l))
print(len(classtoclean_l_catid))

>> Make sure everything is correct: 1.Drop NaN if exist, but make sure the index is the same for the two! 

80
80


In [115]:
print(classtoclean_l[-1])
print(classtoclean_l_catid[-1])

toothbrush
[90]


## 4. clean classes

In [116]:
# slow # newjson=copy.deepcopy(molajson) #do deepcopy to compare
# fast
newjson={'categories':[],'annotations':[] }
newjson['categories']=copy.copy(molajson['categories'])
newjson['annotations']=copy.copy(molajson['annotations'])

In [117]:
classtoclean_l_catidx=[[cat_l_id.index(id) for id in id_l] for id_l in classtoclean_l_catid]
#print(classtoclean_l_catidx) # they should be less one, becacuse it is ordered
print(len(classtoclean_l_catidx))

80


#### Change molajson['categories']: [{name: , id: }]  
=>  1. use first index cat id; 2. change name and change id;  remove the other categories (!!!Without ordering again the category id!!!)

In [118]:
# CHANGE NAME  & GET REMOVE List
keepidx_l=[]
keepid_l=[]
firstidx=0 # get first category id
for i,id_l in enumerate(tqdm(classtoclean_l_catid)): #for each classtoclean
    firstcatid=id_l[firstidx] # #category id 
    firstcatidx=classtoclean_l_catidx[i][firstidx]# get cat index of first catid
    if isinstance(classtoclean_l[i], list): newjson['categories'][firstcatidx]['name']=classtoclean_l[i][firstidx] #change name of first id 
    else: newjson['categories'][firstcatidx]['name']=classtoclean_l[i]
    assert newjson['categories'][firstcatidx]['id']==firstcatid #assert id - it should be the same
    keepidx_l.append(firstcatidx) #catidx to keep
    keepid_l.append(firstcatid) #catid to keep
keepidx_l=list(dict.fromkeys(keepidx_l)) # remove duplicates in the keep list
allidx_l=[index for index, value in enumerate(molajson['categories'])] # allidx in categories
removeidx_l=[idx for idx in allidx_l if idx not in keepidx_l] # remove idx 
removeitem_l=[newjson['categories'][removeidx] for removeidx in removeidx_l] #remove items #WARNING NECESSARY BECAUSE THE INDEX WILL CHANGE
print(len(allidx_l))
print(len(removeidx_l))
print(len(allidx_l)-len(removeidx_l))
print(keepidx_l in removeidx_l)

100%|████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<?, ?it/s]

80
0
80
False


REMOVE CLASSES

In [119]:
# REMOVE - newjson will be changed
for removeitem in removeitem_l: newjson['categories'].remove(removeitem)

In [120]:
print(newjson['categories'][-1])
print(molajson['categories'][-1])

{'supercategory': 'indoor', 'id': 90, 'name': 'toothbrush'}
{'supercategory': 'indoor', 'id': 90, 'name': 'toothbrush'}


REORDER IDs

In [121]:
# GET NEW IDs - REORDER IDs - #WARNING after remove
ct_l_id=[]
for i,c in enumerate(tqdm(newjson['categories'])):
    ct_l_id.append(c['id'])
newidx_l=[ct_l_id.index(id) for id in keepid_l] # make sure same sequence of keepid_l #SAME ORDER AS EXCEL
newid_l=[i+1 for i in range(len(keepid_l))] #reorder keepid_l
print(keepid_l)
print(newidx_l)
print(newid_l)

100%|█████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 79758.57it/s]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]


In [122]:
# SORT IDS - Reorder based on Excel order - newjson will be changed
categories_l=copy.copy(newjson['categories'])
for i,idx in enumerate(newidx_l):
    categories_l[idx]['id']=newid_l[i]
for i,idx in enumerate(newidx_l):
    newjson['categories'][i]=categories_l[idx] #TODO sort the id in the correct sequence

In [123]:
#TEST
print(len(categories_l))
print(len(newjson['categories']))
print(len(molajson['categories']))
print(newjson['categories'][-1])
print(molajson['categories'][-1])

80
80
80
{'supercategory': 'indoor', 'id': 80, 'name': 'toothbrush'}
{'supercategory': 'indoor', 'id': 80, 'name': 'toothbrush'}


### QUESTION: REMOVE HYPERPARAMETERS? Mantain only id and NAME? OR irrelevant?

#### change molajson['annotations']: [{category_id: , }] 
=> 1.get annotation idx from catid; 2.update annotations id ; 3. update newjson['annotations']

In [124]:
# 1.get annotation idx from classtoclean_l_catid
ann_catid_np=np.array(ann_catid)
classtoclean_l_ann_catidx=[[np.where(ann_catid_np==id)[0].tolist()  for id in id_l] for id_l in classtoclean_l_catid]
print(classtoclean_l)
print(classtoclean_l_catid)
print(keepid_l)
print(newid_l)
print(len(classtoclean_l_ann_catidx[0]))

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
[[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [2

In [125]:
#2.update annotations ids & 3. update newjson['annotations'] with only the annotations frow classtoclean_l_catid
newjson['annotations']=copy.copy(molajson['annotations']) #reset annotations
copy_ann_l=copy.copy(newjson['annotations'])
newjson['annotations']=[] #clear
for i, ann_catidx_l in enumerate(classtoclean_l_ann_catidx): #only append annotations that 
    for catidx_l in ann_catidx_l:
        for catidx in catidx_l:
            copy_ann_l[catidx]['category_id']=newid_l[i] # update catid
            newjson['annotations'].append(copy_ann_l[catidx]) #update newjson with only the  (annotations sequence id will be lost
print(len(molajson['annotations']))
print(len(newjson['annotations']))

1170251
1170251


In [126]:
#TEST
print(molajson['annotations'][-1])
print(newjson['annotations'][-1])

{'segmentation': {'counts': [179, 27, 392, 41, 380, 51, 371, 59, 363, 67, 356, 73, 350, 79, 129, 9, 207, 82, 124, 10, 208, 84, 121, 10, 209, 85, 121, 8, 209, 88, 336, 89, 154, 1, 181, 90, 154, 2, 178, 73, 2, 16, 155, 3, 176, 62, 15, 15, 155, 4, 173, 63, 18, 12, 156, 6, 169, 64, 21, 10, 156, 6, 168, 64, 24, 7, 156, 8, 166, 64, 27, 5, 156, 9, 163, 65, 189, 10, 161, 65, 189, 12, 159, 65, 190, 12, 157, 66, 191, 13, 130, 24, 1, 67, 191, 13, 126, 95, 192, 14, 122, 98, 192, 15, 119, 99, 193, 15, 117, 100, 194, 15, 114, 103, 194, 16, 109, 106, 195, 16, 106, 109, 195, 16, 104, 111, 195, 17, 101, 113, 195, 17, 99, 115, 195, 17, 95, 119, 195, 17, 92, 122, 195, 17, 90, 124, 192, 20, 88, 126, 190, 22, 86, 128, 187, 25, 85, 129, 185, 27, 84, 130, 157, 55, 83, 132, 148, 63, 81, 135, 144, 67, 80, 136, 32, 1, 110, 67, 79, 138, 30, 3, 92, 9, 8, 67, 78, 140, 28, 4, 88, 15, 6, 67, 77, 143, 25, 6, 86, 18, 4, 67, 76, 146, 21, 8, 86, 21, 1, 67, 76, 149, 15, 12, 84, 90, 75, 177, 83, 91, 75, 178, 81, 92, 74, 1

### 4. Save cleaned json

In [128]:
# fast
molajson['categories']=copy.copy(newjson['categories'])
molajson['annotations']=copy.copy(newjson['annotations'])

In [129]:
# save
print('\n >> SAVING...')
jsonfile=rdir+injsonfile+'_clean_aggressive.json'
with open(jsonfile, 'w') as f:
    json.dump(molajson, f)
print("JSON SAVED : {} \n".format(jsonfile))
for k in molajson:
    print(k, len(molajson[k]))


 >> SAVING...
JSON SAVED : D:/external_datasets/MOLA/coco2017_mix_aggressive.json 

info 6
licenses 8
images 123287
annotations 1170251
categories 80


### 5. TEST clean ANNOTATIONS DUPLICATES

In [11]:
molajson = json.load(open(rdir+'annotations/split_mola_fix_equal_reorder_nomissings/clean_aggressive_addremainclasses/test.json'))

In [12]:
for k in molajson:
    print(k, len(molajson[k]))

info 5
licenses 9
categories 353
videos 1488
images 177936
tracks 8132
segment_info 0
annotations 133228
datasets 2


In [13]:
# annotations category_id
ann_ids=[]
for an in tqdm(molajson['annotations']):
    ann_ids.append(an['id'])
print(len(ann_ids))

#TEST duplicates v3 -faster
u, c = np.unique(np.array(ann_ids), return_counts=True)
duplicates_l= u[c > 1].tolist()
print(len(duplicates_l))

100%|███████████████████████████████████████████████████████████████████| 133228/133228 [00:00<00:00, 1017127.63it/s]


133228
0


In [14]:
# categories name and id
cat_l=[]
for c in molajson['categories']:
    cat_l.append(c['name'])

In [15]:
print(len(cat_l))
print(cat_l)


353
['aggressive', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'aerosol_can', 'apricot', 'armchair', 'atomizer', 'ax', 'baby_buggy', 'bagpipe', 'ball', 'balloon', 'barbell', 'baseball_bat', 'baseball_glove', 'basketball_hoop'